In [ ]:
import requests
from bs4 import BeautifulSoup

from datetime import datetime
import json
import os

### Fungsi save data ke file JSON

In [7]:

def savejson(new_data, json_file):
    # Cek apakah file sudah ada
    if os.path.exists(json_file):
        try:
            # Buka dan baca isi file JSON
            with open(json_file, "r", encoding="utf-8") as file:
                data = json.load(file)  
                if not isinstance(data, list):  # Pastikan formatnya list
                    data = []
        except (json.JSONDecodeError, OSError) as e:
            print(f"⚠️ Terjadi kesalahan saat membaca {json_file}: {e}")
            data = []  # Jika file rusak atau error, buat list baru
    else:
        data = []  # Jika file belum ada, buat list baru

    # Tambahkan data baru ke dalam list
    data.append(new_data)

    try:
        # Simpan kembali ke file JSON
        with open(json_file, "w", encoding="utf-8") as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        print(f"✅ Data dengan judul '{new_data['title']}' berhasil disimpan ke {json_file}")
    except OSError as e:
        print(f"❌ Gagal menyimpan data ke {json_file}: {e}")


In [ ]:
def get_data(url, json_file):
    
    # Simulasi browser agar tidak diblokir
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    # Request ke halaman berita
    response = requests.get(url, headers=headers)

    # Cek jika request berhasil
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # Mengambil isi artikel dari <div id="zoomthis">
        content_element = soup.find("div", id="zoomthis")
        content = content_element.get_text(separator="\n").strip() if content_element else "Isi artikel tidak ditemukan"
        
        raw = content.split("\n")
        
        #  remove all '' in raw
        while '' in raw:
            raw.remove('')
        

        date = raw[0]
        title = raw[1]
        text = ''.join(raw[2:]).strip()
        
        new_data = {
            "date": date,
            "title": title,
            "text": text,
            "url": url
        }
        savejson(new_data, json_file)
        # print()

    else:
        print(f"Request gagal dengan status: {response.status_code}")

In [5]:
# get_data("http://www.iqplus.info/news/stock_news/ades-ades-catat-pertumbuhan-kinerja-di-kuartal-iii-2023,30340674.html")

In [ ]:
# Open the file and read line by line
with open("stock_news_links.txt", "r", encoding="utf-8") as file:
    for line in file:
        # print(line.strip())  # Strip removes any extra newlines or spaces
        get_data(line.strip())


✅ Data dengan judul 'PUDJI HARIANTO TAMBAH KEPEMILIKAN SAHAM UFOE.' berhasil disimpan ke articles.json
✅ Data dengan judul 'UNILEVER INDONESIA (UNVR) AKAN BAGIKAN DIVIDEN INTERIM PADA 19 DESEMBER 2024.' berhasil disimpan ke articles.json
✅ Data dengan judul 'DIREKTUR UTAMA PT TIMAH TEGASKAN KOMITMEN DALAM IMPLEMENTASIKAN GOOD MINING PRACTICES.' berhasil disimpan ke articles.json
✅ Data dengan judul 'MD ENTERTAINMENT (FILM) PERKUAT MODAL KE ANAK USAHANYA.' berhasil disimpan ke articles.json
✅ Data dengan judul 'PGN TANDA TANGANI PJB LNG DENGAN INDUSTRI DI KAWASAN TIMUR INDONESIA.' berhasil disimpan ke articles.json
✅ Data dengan judul 'JASA MARGA RAIH PENGHARGAAN EMAS DALAM AJANG SNI AWARD 2024.' berhasil disimpan ke articles.json
✅ Data dengan judul 'PEFINDO TETAPKAN OBLIGASI BANK BJB DI PERINGKAT idA+.' berhasil disimpan ke articles.json
✅ Data dengan judul 'TERKAIT VOLATILITAS SAHAMNYA, INI JAWABAN MANAJEMEN SULI KE BEI.' berhasil disimpan ke articles.json
✅ Data dengan judul 'WIKA S

In [13]:
import requests
from bs4 import BeautifulSoup
import time
import random

def get_data(url, json_file):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    # Retry mechanism for handling connection issues
    for attempt in range(5):  # Retry up to 5 times
        try:
            # Request ke halaman berita
            response = requests.get(url, headers=headers, timeout=10)  # Add timeout to avoid hanging
            
            # Cek jika request berhasil (status code 200)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")

                # Mengambil isi artikel dari <div id="zoomthis">
                content_element = soup.find("div", id="zoomthis")
                if not content_element:
                    print(f"⚠️ Konten tidak ditemukan untuk URL: {url}")
                    return

                content = content_element.get_text(separator="\n").strip()

                # Split the content into lines
                raw = content.split("\n")
                
                # Remove empty lines
                raw = [line for line in raw if line.strip()]

                if len(raw) < 3:
                    print(f"⚠️ Data tidak lengkap di {url}")
                    return

                date = raw[0]
                title = raw[1]
                text = ' '.join(raw[2:]).strip()

                new_data = {
                    "date": date,
                    "title": title,
                    "text": text,
                    "url": url
                }
                
                savejson(new_data, json_file)
                print(f"✅ Data berhasil disimpan untuk: {title}")
                return  # Stop retrying if successful

            else:
                print(f"❌ Request gagal untuk {url} dengan status: {response.status_code}")
                return  # Stop trying if the server rejects the request

        except requests.exceptions.RequestException as e:
            print(f"⚠️ Koneksi gagal untuk {url} (Percobaan {attempt + 1}/5): {e}")
            time.sleep(random.uniform(2, 5))  # Wait before retrying

    print(f"❌ Gagal mengambil data setelah 5 percobaan: {url}")


In [15]:
# Open the file and read line by line
with open("market_news_links_master.txt", "r", encoding="utf-8") as file:
    for line in file:
        # print(line.strip())  # Strip removes any extra newlines or spaces
        get_data(line.strip(), "NEW_articles_market_news.json")


✅ Data dengan judul 'HUTAMA KARYA PASTIKAN PEMBANGUNAN TOL SEKSI 4 DI JAMBI SESUAI TARGET.' berhasil disimpan ke NEW_articles_market_news.json
✅ Data berhasil disimpan untuk: HUTAMA KARYA PASTIKAN PEMBANGUNAN TOL SEKSI 4 DI JAMBI SESUAI TARGET.
✅ Data dengan judul 'LPEI DORONG UKM PAPUA MENJADI EKSPORTIR ANDALAN.' berhasil disimpan ke NEW_articles_market_news.json
✅ Data berhasil disimpan untuk: LPEI DORONG UKM PAPUA MENJADI EKSPORTIR ANDALAN.
✅ Data dengan judul 'TRENGGONO DUKUNG SWASEMBADA PANGAN DENGAN MEMASTIKAN SISI HULU.' berhasil disimpan ke NEW_articles_market_news.json
✅ Data berhasil disimpan untuk: TRENGGONO DUKUNG SWASEMBADA PANGAN DENGAN MEMASTIKAN SISI HULU.
✅ Data dengan judul 'KENAIKAN HARGA BITCOIN TINGKATKAN KEPERCAYAAN INVESTOR ASET KRIPTO.' berhasil disimpan ke NEW_articles_market_news.json
✅ Data berhasil disimpan untuk: KENAIKAN HARGA BITCOIN TINGKATKAN KEPERCAYAAN INVESTOR ASET KRIPTO.
✅ Data dengan judul 'PERTAMINA SMEXPO 2024 CATAT TRANSAKSI HINGGA Rp17,45 MILI